In [29]:
from llama_index.core import SimpleDirectoryReader, Settings, StorageContext
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.supabase import SupabaseVectorStore
from supabase import create_client
import tempfile, shutil, pathlib

In [15]:
# --- temp-copy the user PDF -----------------------------
pdf_path = "/Users/anwarmujeeb/Development/CareFlow/shared/Post-Surgical Care Plan_ Knee Arthroscopy.pdf"  # already uploaded
tmpdir   = tempfile.mkdtemp()
shutil.copy(pdf_path, tmpdir)


'/var/folders/5t/613b1kv54272dgj3mkm5qts00000gn/T/tmp4ba0lx2t/Post-Surgical Care Plan_ Knee Arthroscopy.pdf'

In [23]:
client = create_client(os.getenv("SUPABASE_URL"),
                       os.getenv("SUPABASE_SERVICE_ROLE_KEY"))

vec_store = SupabaseVectorStore(
    postgres_connection_string=postgres_connection_string,  # Required parameter
    collection_name="care_plans_collection",               # Required parameter
    client=client,                                         # Optional: your existing client
    table_name="care_plans"                                # Optional: your table name
)

In [30]:
# --- Configure storage context ---
storage_context = StorageContext.from_defaults(vector_store=vec_store)

# --- parse the PDF & build the index --------------------
docs = SimpleDirectoryReader(tmpdir).load_data()   # LlamaIndex auto-splits
index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context,
    # No need to pass service_context as we've set the Settings globally
)
shutil.rmtree(tmpdir)          # clean-up
print(f"Inserted {len(docs)} chunks into Supabase!")

Inserted 6 chunks into Supabase!


In [31]:
# --- Test the index with a query engine ---
query_engine = index.as_query_engine()

# Ask a question about knee arthroscopy recovery
response = query_engine.query("What activities should I avoid after knee arthroscopy?")
print("Response:")
print(response.response)

# Ask about pain management
response = query_engine.query("How should I manage pain after knee arthroscopy surgery?")
print("\nResponse about pain management:")
print(response.response)

# Ask about post-surgical rehabilitation exercises
response = query_engine.query("What exercises are recommended during rehabilitation?")
print("\nResponse about rehabilitation exercises:")
print(response.response)

# Ask about potential complications
response = query_engine.query("What complications should I watch out for after surgery?")
print("\nResponse about complications:")
print(response.response)

/opt/homebrew/Caskroom/miniforge/base/envs/temp/lib/python3.12/site-packages/vecs/collection.py:506: UserWarning: Query does not have a covering index for IndexMeasure.cosine_distance. See Collection.create_index
  warnings.warn(


Response:
No weight-bearing on the surgical leg, keeping the surgical dressing clean and dry, and avoiding showering are activities that should be avoided after knee arthroscopy.

Response about pain management:
Manage pain after knee arthroscopy surgery by taking Tylenol as needed, not exceeding 3000mg in 24 hours. Additionally, Ibuprofen can be taken with food as needed for pain and swelling. It is important to follow the prescribed medication schedule and dosage instructions provided by the healthcare provider.

Response about rehabilitation exercises:
Quad sets, straight leg raises, seated knee flexion, hamstring stretches, standing weight shifts, gentle mini-squats, and walking without crutches for short distances inside the home are recommended exercises during rehabilitation.

Response about complications:
Fever over 101°F, excessive pain not controlled by prescribed medications, calf pain, swelling, or tenderness, excessive drainage or bleeding from incision sites, increased re